**PART 3**
Exploring and clustering the neighborhoods in Sengkang


# Introduction
I will be using the Foursquare API to explore the nearby venues in SengKang 
<br>
I shall use the Folium library to visualize the various venues in Sengkang 

In [1]:
# Importing the required libraries
import numpy as np 
import pandas as pd 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 


# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
GeoLocator = Nominatim(user_agent='My-IBMNotebook')

 # library to handle requests and transform JSON file into a Pandas DataFrame
import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
# Retrieving the data that was cleaned up in Part 2 of this assignmentHDB_2/Sengkang_Part_2.csv")
sk_neighborhoods = pd.read_csv("C:/Users/Buck Hui/Documents/IBM Data Science/^ Capstone Project/HDB_2/Sengkang_Part_2.csv")
sk_neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,SENGKANG,ANCHORVALE CRES,103.889866,1.397981
1,SENGKANG,ANCHORVALE DR,103.891536,1.390196
2,SENGKANG,ANCHORVALE LANE,103.884412,1.392105
3,SENGKANG,ANCHORVALE LINK,103.890182,1.392388
4,SENGKANG,ANCHORVALE RD,103.888536,1.394942


In [3]:
sk_neighborhoods.shape


(29, 4)

In [4]:
address = 'Sengkang, Singapore'

geolocator = Nominatim(user_agent="sengkang_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sengkang, Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sengkang, Singapore are 1.3919236499999998, 103.89549093760694.


In [5]:
# create map of SK using latitude and longitude values
map_sk = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(sk_neighborhoods['Latitude'], sk_neighborhoods['Longitude'], sk_neighborhoods['Borough'], sk_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_sk)
    
map_sk

In [6]:
sk_data = sk_neighborhoods[sk_neighborhoods['Borough'].str.contains("SENGKANG")].reset_index(drop=True)
print(sk_data.shape)
sk_data.head()

(29, 4)


,Borough,Neighbourhood,Latitude,Longitude
0,SENGKANG,ANCHORVALE CRES,103.889866,1.397981
1,SENGKANG,ANCHORVALE DR,103.891536,1.390196
2,SENGKANG,ANCHORVALE LANE,103.884412,1.392105
3,SENGKANG,ANCHORVALE LINK,103.890182,1.392388
4,SENGKANG,ANCHORVALE RD,103.888536,1.394942


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(sk_neighborhoods['Borough'].unique()),
        sk_neighborhoods.shape[0]
    )
)

The dataframe has 1 boroughs and 29 neighborhoods.


In [8]:
CLIENT_ID = 'redacted for confidentiality' # your Foursquare ID
CLIENT_SECRET = 'redacted for confidentiality' # your Foursquare Secret
VERSION = '20201115'
LIMIT = 30

In [9]:
neighborhood_latitude = 1.3919236499999998 # neighborhood latitude value
neighborhood_longitude = 103.89549093760694 # neighborhood longitude value

#1.3919236499999998, 103.89549093760694.

In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=K3FRWPPRTT2O4WLNKQGL3VHKEBT5DFBMHKNAWJIR4LSCOC2Q&client_secret=UAGPJ0GYP1TWL03VKXW3COJQ0SXEUOIA40LOGWKJDSTTFRXG&v=20201115&ll=1.3919236499999998,103.89549093760694&radius=500&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc4b53d72ff46632005f1fe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Sengkang',
  'headerFullLocation': 'Sengkang, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 1.3964236545000044,
    'lng': 103.89998386583989},
   'sw': {'lat': 1.3874236454999953, 'lng': 103.89099800937399}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57c90306498e61437fbdac58',
       'name': 'Maki-san',
       'location': {'address': '#01-45, Compass One',
        'crossStreet': '1 Sengkang Sq.',
        'lat': 1.3923110109254946,
        'lng': 103.8949687237786,
        'labeledLatLngs': [{'l

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-13-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Maki-san,Sushi Restaurant,1.392311,103.894969
1,Ya Kun Family Cafe,Café,1.391903,103.894913
2,Starbucks,Coffee Shop,1.392367,103.895018
3,Châteraisé,Bakery,1.392245,103.895079
4,Canton Paradise,Chinese Restaurant,1.392299,103.895128


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

29 venues were returned by Foursquare.


In [15]:
print(nearby_venues.shape)
nearby_venues.head()

(29, 4)


,name,categories,lat,lng
0,Maki-san,Sushi Restaurant,1.392311,103.894969
1,Ya Kun Family Cafe,Café,1.391903,103.894913
2,Starbucks,Coffee Shop,1.392367,103.895018
3,Châteraisé,Bakery,1.392245,103.895079
4,Canton Paradise,Chinese Restaurant,1.392299,103.895128


# Count of venues were returned for each Borough

In [16]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
Asian Restaurant,1,1,1
Bakery,1,1,1
Bubble Tea Shop,1,1,1
Bus Station,1,1,1
Café,1,1,1
Chinese Restaurant,1,1,1
Coffee Shop,2,2,2
Cosmetics Shop,2,2,2
Electronics Store,1,1,1


# How many unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 24 uniques categories.


# To visualize the clusters

In [18]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to the map
markers_colors = []
for lat, lng, cat in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters